In [1]:
%pip install openai tiktoken chromadb langchain esprima faiss-cpu unstructured
#%pip install langchain -U




[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import dotenv
import os
import openai

dotenv.load_dotenv()


True

In [2]:
# from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.directory import DirectoryLoader
from langchain.document_loaders.parsers.language import LanguageParser
from langchain.document_loaders.parsers.txt import TextParser
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader


In [ ]:
# Clone

# repo = Repo.clone_from("https://github.com/HgDataTeam/hg-gpt-similarity.git", to_path=repo_path)


In [3]:
# Load
repo_path = "/Users/aren.blondahl/stash/wpg"
#loader = GenericLoader.from_filesystem(
#    repo_path,
#    glob="**/*.{php|inc|module|test}",
#    parser=LanguageParser(language=Language.PHP, parser_threshold=50)
#)

#loader = GenericLoader.from_filesystem("./php_code", glob="*.php", parser=LanguageParser(language="php"))

file_loader_args = {
    # 'blob_loader': FileSystemBlobLoader,
    'blob_parser': LanguageParser(language=Language.PHP, parser_threshold=50)
}

#loader = DirectoryLoader(
#    repo_path, recursive=True, glob="sites/**/*[!_mailerTest].php",
#    loader_cls=GenericLoader,
#    loader_kwargs=file_loader_args
#)

loader = GenericLoader.from_filesystem(
    repo_path,
    glob="sites/**/*[!_mailerTest].php",
    parser=TextParser()
    #parser=LanguageParser(language=Language.PHP, parser_threshold=50)
)

documents = loader.load()
len(documents)

1529

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PHP, 
                                                               chunk_size=4000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

2485

In [12]:
openai_api_base = os.getenv('OPENAI_API_BASE')
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_api_type = os.getenv('OPENAI_API_TYPE')
openai_api_version = os.getenv('OPENAI_API_VERSION')
openai_deployment = os.getenv('OPENAI_API_DEPLOYMENT_NAME')
openai_deployment_embeddings = os.getenv('OPENAI_API_EMBEDDINGS_NAME')

#from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(disallowed_special=(), 
                                                   deployment=openai_deployment_embeddings,
                                                   openai_api_base=openai_api_base, 
                                                   openai_api_key=openai_api_key, 
                                                   openai_api_type=openai_api_type,
                                                   openai_api_version=openai_api_version, chunk_size=16)

#output_dir = "./db_metadata_v5"

# generate text embeddings for our target codebase
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

EngTextEmbedding


In [10]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
llm = AzureChatOpenAI(deployment_name=openai_deployment,
                          openai_api_base=openai_api_base,
                          openai_api_version=openai_api_version,
                          openai_api_key=openai_api_key,
                          openai_api_type=openai_api_type,
                          temperature=0.7,
                          verbose=True)
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "From which external infrastructure components does the service depend?"
result = qa(question)
print(result['answer'])

In [11]:
#add some code to in a loop ask for additional questions and print the answers
while True:
    question = input("Ask a question: ")
    if question == "quit":
        break
    print("Question: ", question)

    result = qa(question)
    print("Answer: ", result['answer'])

Question:  Explain to me how payroll works
Answer:  The code snippets provided are quite technical and specific to a particular payroll system implementation. However, I can provide a general explanation of how payroll systems usually work.

1. Employee Information: The payroll process begins with collecting information about employees, their work hours, and the agreed wage rate. This includes their tax information, social security, and other details.

2. Calculation: The next step involves calculating the gross pay. For hourly employees, this is typically done by multiplying the hours worked by the wage rate. For salaried employees, the salary might be divided across the pay periods. 

3. Deductions: From the gross pay, deductions are subtracted. These can include tax withholdings (based on the employee's W-4 form information), social security and Medicare contributions, health insurance premiums, retirement contributions, and any other deductions like garnishments.

4. Net Pay: After

KeyboardInterrupt: 